Pytorch处理MNIST示例
Mac安装命令为：

    pip install torch torchvision


In [1]:
import os
import torch
import torchvision
from torch.autograd import Variable
import torch.utils.data.dataloader as Data
import torch.nn.functional as F
from torchvision import datasets, transforms

train_data = torchvision.datasets.MNIST(
 'dataset/mnist-pytorch', train=True, 
    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])]),
    download=True
)
test_data = torchvision.datasets.MNIST(
 'dataset/mnist-pytorch', train=False, 
    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])
)
print("train_data:", train_data.train_data.size())
print("train_labels:", train_data.train_labels.size())
print("test_data:", test_data.test_data.size())

#批大小
batch_size=128
#训练的批次数
epochs=20

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.dense = torch.nn.Sequential(
            torch.nn.Linear(784, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 10),
            torch.nn.ReLU()
        )


    def forward(self, x):
        x = x.view(-1, 784)
        x=self.dense(x)
        return torch.nn.functional.log_softmax(x, dim=1)

def main():

    # 自适应使用GPU还是CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Net().to(device)

    print(model)
    
    optimizer = torch.optim.Adam(model.parameters())
    loss_func = torch.nn.CrossEntropyLoss()


    train_loader = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    test_loader = Data.DataLoader(dataset=test_data, batch_size=batch_size)

    for epoch in range(epochs):
        sum_loss = 0.0
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # 梯度清零
            optimizer.zero_grad()

            # forward + backward
            outputs = model(inputs)

            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            # 每训练100个batch打印一次平均loss
            sum_loss += loss.item()
            if (i+1) % 100 == 0:
                print('epoch=%d, batch=%d loss: %.04f'
                      % (epoch + 1, i+1, sum_loss / 100))
                sum_loss = 0.0
        # 每跑完一次epoch测试一下准确率 进入测试模式 禁止梯度传递
        with torch.no_grad():
            correct = 0
            total = 0
            for data in test_loader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                # 取得分最高的那个类
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
            print('epoch=%d accuracy=%.02f%%' % (epoch + 1, (100 * correct / total)))

main()

('train_data:', torch.Size([60000, 28, 28]))
('train_labels:', torch.Size([60000]))
('test_data:', torch.Size([10000, 28, 28]))
Net(
  (dense): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=10, bias=True)
    (4): ReLU()
  )
)
epoch=1, batch=100 loss: 0.6037
epoch=1, batch=200 loss: 0.2501
epoch=1, batch=300 loss: 0.1904
epoch=1, batch=400 loss: 0.1635
epoch=1 accuracy=96.00%
epoch=2, batch=100 loss: 0.1173
epoch=2, batch=200 loss: 0.1123
epoch=2, batch=300 loss: 0.0940
epoch=2, batch=400 loss: 0.0963
epoch=2 accuracy=97.00%
epoch=3, batch=100 loss: 0.0665
epoch=3, batch=200 loss: 0.0690
epoch=3, batch=300 loss: 0.0673
epoch=3, batch=400 loss: 0.0721
epoch=3 accuracy=97.00%
epoch=4, batch=100 loss: 0.0444
epoch=4, batch=200 loss: 0.0447
epoch=4, batch=300 loss: 0.0469
epoch=4, batch=400 loss: 0.0497
epoch=4 a